### Setup

In [3]:
!cd /content
!gdown https://drive.google.com/uc?id=1ahp0Ac3I_7Fm1usqPT_Rvgww5KaLvo6Y
!unzip davis.zip; rm davis.zip

Downloading...
From: https://drive.google.com/uc?id=1ahp0Ac3I_7Fm1usqPT_Rvgww5KaLvo6Y
To: /content/davis.zip
4.32MB [00:00, 138MB/s]
Archive:  davis.zip
replace davis/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [4]:
!git clone https://github.com/rohaldb/multi-mono-sf.git
%cd multi-mono-sf
!chmod -R +x ./
!pip install -r requirements.txt

Cloning into 'multi-mono-sf'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 93 (delta 21), reused 83 (delta 18), pack-reused 0
Unpacking objects: 100% (93/93), done.
/content/multi-mono-sf
     |████████████████████████████████| 61kB 5.5MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


### Demo
Run the following and then look in `eval/davis/rollerblade`

In [ ]:
%cd /content/multi-mono-sf/scripts
!./eval_davis.sh